In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [4]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=transforms.ToTensor()
)
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=transforms.ToTensor()
)

In [16]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [29]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
        
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    
model = Net().to(device)
print(model)

Net(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [30]:
loss_fn = nn.CrossEntropyLoss() # difference of the measure between two probability distributions
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3) # lr will be the 1st parameter to tune when training

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
        
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.inference_mode(): # more modern and descriptive than torch.no_grad()
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [31]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.303004  [    0/60000]
loss: 0.567371  [ 6400/60000]
loss: 0.393993  [12800/60000]
loss: 0.498452  [19200/60000]
loss: 0.467649  [25600/60000]
loss: 0.419780  [32000/60000]
loss: 0.363314  [38400/60000]
loss: 0.519002  [44800/60000]
loss: 0.480197  [51200/60000]
loss: 0.522765  [57600/60000]
Test Error: 
 Accuracy: 84.2%, Avg loss: 0.423269 

Epoch 2
-------------------------------
loss: 0.268192  [    0/60000]
loss: 0.357002  [ 6400/60000]
loss: 0.270909  [12800/60000]
loss: 0.379903  [19200/60000]
loss: 0.433206  [25600/60000]
loss: 0.382488  [32000/60000]
loss: 0.317162  [38400/60000]
loss: 0.452083  [44800/60000]
loss: 0.392341  [51200/60000]
loss: 0.433317  [57600/60000]
Test Error: 
 Accuracy: 85.0%, Avg loss: 0.399751 

Epoch 3
-------------------------------
loss: 0.227462  [    0/60000]
loss: 0.339121  [ 6400/60000]
loss: 0.229746  [12800/60000]
loss: 0.316768  [19200/60000]
loss: 0.394329  [25600/60000]
loss: 0.365011  [32000/600

In [32]:
torch.save(model.state_dict(), "mnist_fashion_base_model.pth") # save the weights

In [36]:
model = Net().to(device)
model.load_state_dict(torch.load("mnist_fashion_base_model.pth"))
model.eval()

from PIL import Image

image_path = "test_tshirt.png"
image = Image.open(image_path).convert("L") # convert to grayscale
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((28, 28)),
])
image = transform(image).to(device) # add batch dimension

with torch.inference_mode():
    output = model(image.unsqueeze(0)) # add batch dimension
    predicted_label = output.argmax(1).item()
    print(f"Predicted label: {predicted_label}")

Predicted label: 0
